# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 1 ноября 2018, 06:00 
**Штраф за опоздание:** -2 балла после 06:00 1 ноября, -4 балла после 06:00 8 ноября, -6 баллов после 06:00 15 ноября

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (4 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы fit такой, чтобы она была медленнее sklearn не более чем в 10 раз. Скорость проверяем на  wine и Speed Dating Data. Для ускорения используем только numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 3 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [22]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%reload_ext pycodestyle_magic

In [29]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - np.sum((l_c ** 2) / (l_s * (l_s + r_s)) + (
            r_c ** 2) / (r_s * (l_s + r_s)), axis=1)

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -np.sum((l_c / l_s).log2() * l_c / (l_s + r_s) + (
            r_c / r_s).log2() * r_c / (l_s + r_s), axis=1)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - np.max(l_c / (l_s + r_s), axis=1) - np.max(
            r_c / (l_s + r_s), axis=1)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.sqrt(n_feature))])

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.log2(n_feature))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Сортируются x и y по признакам и получает количество классов.
        # Находится class_number - количество классов.
        # Вычисляется cut_size - число,
        # на которое нужно разрезать массив признаков х,
        # чтобы продолжить разбиение,
        # равное половине минимального числа элементов в вершине - 1(она сама).
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)

        # В вырезанном отрезке (слева и справа вырезали cut_size элементов)
        # находятся индексы, для которых меняется номер класса в y,
        # если их нет, то возвращается None.
        # Прибавляется индекс, с которого начинается вырезанный отрезок.
        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)

        if len(r_border_ids) == 0:
            return np.inf, None

        # Находится eq_el_count - количество одинаковых элементов в интревале.
        # Создаётся one_hot_code - нулевая матрица для количества классов.
        # В этой матрице ставятся единицы на месте тех классов,
        # в которых происходит смена.
        # Создаётся class_hot_increment - матрица количества элементов
        # в каждом интервале до очередной смены классов.
        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)

        # Считается сколько элементов принадлежат каждому классу
        # в левой и правой части разбиения.
        # Считаются l_size,r_size - количества классов
        # в левой и правой части разбиения.
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Считается мера неопределённость.
        # Вычисляется аргумент, при котором найдйденная мера минимальна.
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Выбирается номер, начиная с которого надо вырезать.
        # Возвращается минимальная мера неопределённости
        # и порог, начиная с которого надо разделять.
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        if (self.max_depth is not None and self.max_depth <= depth) or\
                y.size < self.min_samples_split or\
                self.sufficient_share <= np.bincount(y).argmax() / y.size:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
        else:
            threshold = np.array([self.__find_threshold(x[:, i], y)
                                  for i in self.get_feature_ids(x.shape[1])])
            best_id = threshold[:, 0].argmin()
            best_threshold = threshold[best_id, 1]
            if best_threshold is None:
                self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                      np.bincount(y).astype(float) / y.size)
            else:
                x_l, x_r, y_l, y_r =\
                    self.__div_samples(x, y, best_id, best_threshold)
                if x_l.size == 0 or x_r.size == 0:
                    self.tree[node_id] = (self.LEAF_TYPE,
                                          np.bincount(y).argmax(),
                                          np.bincount(y).
                                          astype(float) / y.size)
                else:
                    self.tree[node_id] = (self.NON_LEAF_TYPE,
                                          best_id, best_threshold)
                    self.__fit_node(x_l, y_l,
                                    2 * node_id + 1, depth + 1, pred_f)
                    self.__fit_node(x_r, y_r,
                                    2 * node_id + 2, depth + 1, pred_f)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [30]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [31]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [32]:
%time clf.fit(X_train, y_train)

Wall time: 100 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [33]:
%time my_clf.fit(X_train, y_train)

Wall time: 261 ms


## Проверка качества работы на wine

In [34]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

1.0

In [35]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

1.0

## Подготовка данных Speed Dating Data 

In [37]:
df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')
df = df.iloc[:, :97]
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df.drop_duplicates(subset=['iid']).gender.value_counts()
df = df.drop(['condtn'], axis=1)
df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)
df = df.drop(['age_o', 'race_o', 'pf_o_att',
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o'],
             axis=1)
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df = df.drop(['field'], axis=1)
pd.get_dummies(df, columns=['field_cd'],
               prefix='field_cd', prefix_sep='=')
df = df.drop(['undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', 
                                                    '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df = df.drop(['career'], axis=1)
df = df.drop(['sports', 'tvsports', 'exercise',
              'dining', 'museums', 'art', 'hiking',
              'gaming', 'clubbing', 'reading', 'tv',
              'theater', 'movies', 'concerts',
              'music', 'shopping', 'yoga'], axis=1)
df = df.drop(['expnum'], axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:, [
    'attr1_1', 'sinc1_1', 'intel1_1',
    'fun1_1', 'amb1_1', 'shar1_1'
]].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1',
           'intel1_1', 'fun1_1',
           'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1',
                'intel1_1', 'fun1_1',
                'amb1_1', 'shar1_1']].T / df.loc[:, 'temp_totalsum'].T).T * 100
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1',
                                        'intel2_1', 'fun2_1',
                                        'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1',
           'intel2_1', 'fun2_1',
           'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
                'amb2_1', 'shar2_1']].T / df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]

    if i != 4:
        feat.remove('shar{}_1'.format(i))

    df = df.drop(feat, axis=1)
df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) \
    .drop(['gender'], axis=1) \
    .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) \
    .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1) \
    .dropna()

df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'),
                       on='pid',
                       how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1)
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
my_clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [38]:
%time clf.fit(X_train, y_train)

Wall time: 375 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [39]:
%time my_clf.fit(X_train, y_train)

Wall time: 340 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

## Проверка качества работы на Speed Dating Data

In [40]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5310899690272585

In [41]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5113463887757747

## Задание 3

In [42]:
print(pd.Series(data=clf.feature_importances_, index=df_pair.columns[1:]).sort_values(ascending=False).head(10))
print(pd.Series(data=my_clf.feature_importances_, index=df_pair.columns[1:]).sort_values(ascending=False).head(10))

int_corr     0.071706
amb1_1       0.030994
attr2_1      0.030732
attr2_1_f    0.030066
fun2_1_f     0.030039
amb3_1_f     0.027454
income_f     0.024608
sinc1_1      0.022867
fun1_1_f     0.021377
age          0.021252
dtype: float64
int_corr     0.088005
sinc1_1      0.030407
amb1_1       0.027517
attr2_1      0.027348
attr2_1_f    0.026878
fun2_1_f     0.026350
age          0.024700
fun1_1_f     0.024631
attr3_1_f    0.020730
mn_sat_f     0.020539
dtype: float64


## Задание 4

In [44]:
p = {
    'max_depth': range(1, 30),
    'min_samples_leaf': range(1, 20),
    'criterion': ['gini', 'entropy'],
    'n_estimators': range(1, 150)
}
clf = RandomizedSearchCV(RandomForestClassifier(),
                          param_distributions=p)
clf.fit(X_train, y_train)
clf.best_params_

{'n_estimators': 113,
 'min_samples_leaf': 1,
 'max_depth': 21,
 'criterion': 'entropy'}